In [1]:
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
import os
import gensim.downloader as api
import string
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import words
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import gensim.downloader as api
import re
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [2]:
# will be used to analyse all three dataframes 
word_vectors = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [5]:
blogdf = pd.read_csv('/content/drive/MyDrive/text_mining_project/naturalscience/attempt2.csv')

words = pd.read_csv('/content/drive/MyDrive/text_mining_project/naturalscience/facktermer_naturvetenskap.csv') 

words_biology = words[words['theme'] == 'Biology']['words'].unique().tolist()
words_physics = words[words['theme'] == 'Physics']['words'].unique().tolist()
words_chemistry = words[words['theme'] == 'Chemistry']['words'].unique().tolist()

blogdf

,Unnamed: 0,Id,Comment,Topic,cleaned_Comments
0,0,0x840,A few things. You might have negative- frequen...,Biology,thing you might negative frequency dependent s...
1,1,0xbf0,Is it so hard to believe that there exist part...,Physics,hard believe exist particular detect anything ...
2,2,0x1dfc,There are bees,Biology,there bee
3,3,0xc7e,I'm a medication technician. And that's alot o...,Biology,medication technician and alot drug liver you ...
4,4,0xbba,Cesium is such a pretty metal.,Chemistry,cesium pretty metal
...,...,...,...,...,...
10276,1581,0x22bf,I’m not really denying your intent. I’m a) dou...,Chemistry,really denying intent doubting third comment i...
10277,1582,0x1f4a,i really empathize with your compassion for bu...,Biology,really empathize compassion bug entomology rea...
10278,1583,0x27b7,"If you want to keep it for more than 2 weeks, ...",Chemistry,want keep week would something like you also w...
10279,1584,0x2066,Same here. I’m in nursing school so I only hav...,Biology,same nursing school basic understanding thing ...


In [19]:
class Selecting_threegrams: 
  def __init__(self, blogposts): 
    # här sparar jag alla n-grams 
    self.threegram_dict={}  
    self.blogposts = blogposts 

  def store_three_grams(self):  
    nlp = spacy.load("en_core_web_sm")
    throwawaylist = {}
    self.three_gram_dict = { } 


    def ngram(inputwordlist): 
      ngramlist = [(wordlist[l],wordlist[l+1], wordlist[l+2])   for l in range(LW-2)]  

      uniquelist = [ ]
      for ngram in ngramlist:
        if ngram not in uniquelist:
          uniquelist.append(ngram) 

      for ngram in uniquelist: 
        if ngram not in self.threegram_dict:
          self.threegram_dict[ngram] = 1
        else: 
          self.threegram_dict[ngram] += 1

    for blogs in self.blogposts: 
      try:
        wordlist = blogs.split() 
        LW = len(wordlist)
        if LW > 4: 
          ngram(wordlist)
      except: 
        throwawaylist.append(blogs)
    trigrams = [ ]
    for k, v in self.threegram_dict.items(): 
      if v > 3: 
        phrase = ' '.join(k)
        trigrams.append(phrase)
    return trigrams



  def popular_two_grams(self, number, wordlist): 
      newdict = { }
      for k, v in self.threegram_dict.items():  
        word1 = k[0]
        word2 = k[1] 
        word3 = k[2] 
        if word1 in wordlist or word2 in wordlist or word3 in wordlist:
          if v > number: 
            newdict[' '.join(k)] = v

      sorted_list = sorted(newdict.items(), key=lambda x: x[1], reverse=True)  

      self.sorted_list = sorted_list  


In [32]:
blogs = blogdf['Comment'].dropna(how='all')
Biology_blogs = blogdf[blogdf['Topic']=='Biology']['cleaned_Comments'].dropna()

model1 = Selecting_threegrams(Biology_blogs.tolist() )
model1.store_three_grams( )
model1.popular_two_grams(2, words_biology )

In [ ]:
blogs = blogdf['Comment'].dropna(how='all')
Physics_blogs = blogdf[blogdf['Topic']=='Physics']['cleaned_Comments'].dropna()

model1 = Selecting_threegrams(Physics_blogs.tolist() )
model1.store_three_grams( )
model1.popular_two_grams(2, words_physics ) 

In [50]:
blogs = blogdf['Comment'].dropna(how='all')
Chemistry_blogs = blogdf[blogdf['Topic']=='Chemistry']['Comment'].dropna()

model1 = Selecting_threegrams(Chemistry_blogs.tolist() )
model1.store_three_grams( )
model1.popular_two_grams(1, words_chemistry )

In [42]:
class Selecting_ngrams: 
  def __init__(self, blogposts): 
    # här sparar jag alla n-grams 
    self.twogram_dict={}  
    self.blogposts = blogposts 

  def store_two_grams(self):  
    nlp = spacy.load("en_core_web_sm")
    self.twogramdict = { }
    newngramdict = {} 
    throwawaylist = []

    def ngrams(wordlist): 
      ngramlist = [(wordlist[l],wordlist[l+1])   for l in range(LW-1)]  

      for ngram in ngramlist: 
        current_ngram = ' '.join(ngram)
        doc = nlp(current_ngram) 
        pos = [ ]
        for token in doc: 
          # to check the part-of-speech tags
          pos.append(token.pos_)  
        if pos[0] == 'NOUN' and pos[1] == 'NOUN':
          if ngram not in self.twogram_dict:
            self.twogram_dict[ngram] = 1
          else: 
            self.twogram_dict[ngram] += 1

    for blogs in self.blogposts: 
      try:
        wordlist = blogs.split() 
        LW = len(wordlist)
        if LW > 2: 
          ngrams(wordlist)
      except: 
        throwawaylist.append(blogs)



  def popular_two_grams(self, number, wordlist): 
      newdict = { }
      self.included_keywords = []
      for k, v in self.twogram_dict.items():  
        word1 = k[0]
        word2 = k[1] 
        if word1 in wordlist or word2 in wordlist:
          if word1 in wordlist: 
            self.included_keywords.append(word1)
          if word2 in wordlist:
            self.included_keywords.append(word1)
          if v > number: 
            newdict[' '.join(k)] = v

      sorted_list = sorted(newdict.items(), key=lambda x: x[1], reverse=True)  

      self.sorted_list = sorted_list  

      selectedlist = [ ]

      # to create a list over selected list: 
      for elem in sorted_list: 
        selectedlist.append(elem[0])

      return selectedlist

  def create_new_dataframe(self, selected_ngramlist): 
    # to create a new dataframe to store the twograms 
    twogram_list = [ ]
    cleaned_text = [ ] 
    throwawaylist = [ ]  

    def process_blogs(inputlist): 
      ngramlist = [(wordlist[l],wordlist[l+1])   for l in range(LW-1)]  
      uniquelist = []
      for ngram in ngramlist:
        if ngram not in uniquelist:
          uniquelist.append(ngram)

      for ngram in uniquelist: 
        phrase = ' '.join(ngram) 
        if phrase in selected_ngramlist: 
          twogram_list.append(phrase) 
          cleaned_text.append(' '.join(inputlist) )

    for blogs in self.blogposts: 
      try:
        wordlist = blogs.split() 
        LW = len(wordlist)
        if LW > 2: 
          process_blogs(wordlist)
      except: 
        throwawaylist.append(blogs)  

    twogram_df = pd.DataFrame(list(zip(twogram_list, cleaned_text)),
               columns =['ngrams', 'Cleaned_text'])

    return twogram_df
    

In [54]:
blogs = blogdf['cleaned_Comments'].dropna(how='all')
physics_blogs = blogdf[blogdf['Topic']=='Chemistry']['cleaned_Comments'].tolist()

model1 = Selecting_ngrams(physics_blogs) 

model1.store_two_grams( )
listofwords = model1.popular_two_grams( number = 2, wordlist = words_chemistry)
#twogram_df = model1.create_new_dataframe(listofwords3)
dikt_trigram = model1.sorted_list
newtrigramlist = [ ]
for elem in dikt_trigram: 
    newtrigramlist.append(elem[0])

In [58]:
twogram_df = model1.create_new_dataframe(listofwords4)
twogram_df
twogram_df.to_csv('/content/drive/MyDrive/text_mining_project/naturalscience/twograms_chemistry.csv') 